# Chapter 07: QLoRA를 활용한 SFT

## 1. 학습 목표

* QLoRA의 원리와 장점(메모리 절약, 성능 유지)을 이해한다.
* 4-bit 양자화 모델에 LoRA를 적용하여 금융 도메인 데이터를 학습한다.
* `paged_adamw_8bit` 옵티마이저를 활용해 메모리 부족(OOM)을 방지한다.

## 2. QLoRA 설정 및 모델 로드

QLoRA의 핵심은 **4-bit NormalFloat (NF4)** 양자화와 **이중 양자화(Double Quantization)**이다. 이를 통해 20B급 모델을 약 12~14GB VRAM에 로드할 수 있다.

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# optional imports (환경에 따라 없을 수 있음)
try:
    from transformers import BitsAndBytesConfig
except Exception:
    BitsAndBytesConfig = None

try:
    from transformers import Mxfp4Config
except Exception:
    Mxfp4Config = None

try:
    from peft import prepare_model_for_kbit_training
except Exception:
    prepare_model_for_kbit_training = None


def _is_gpt_oss(model_id: str) -> bool:
    mid = (model_id or "").lower()
    return "openai/gpt-oss" in mid or mid.startswith("openai/gpt-oss")


def _make_bnb_nf4_4bit_config():
    if BitsAndBytesConfig is None:
        raise ImportError(
            "BitsAndBytesConfig를 import할 수 없음. "
            "pip install -U bitsandbytes transformers accelerate 를 확인하라."
        )
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

def load_model_and_tokenizer(
    model_id: str,
    *,
    device_map: str = "auto",
    trust_remote_code: bool = True,
    gpt_oss_dequantize_to_bf16: bool = False,  # gpt-oss-20b만 해당
    enable_gradient_checkpointing: bool = True,
):
    """
    - gpt-oss-20b: MXFP4 네이티브 로드(기본) 또는 BF16 디양자화 로드
    - 그 외: bitsandbytes NF4 4bit 로드 + (가능하면) k-bit 학습 준비
    """
    is_oss = _is_gpt_oss(model_id)

    if is_oss:
        if Mxfp4Config is None:
            raise ImportError(
                "Mxfp4Config를 import할 수 없음. "
                "pip install -U transformers 로 업데이트하라."
            )

        quant_cfg = Mxfp4Config(dequantize=bool(gpt_oss_dequantize_to_bf16))
        torch_dtype = torch.bfloat16 if gpt_oss_dequantize_to_bf16 else "auto"

        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map=device_map,
            trust_remote_code=trust_remote_code,
            quantization_config=quant_cfg,
            torch_dtype=torch_dtype,
            # MXFP4 환경에서 sdpa가 안 맞는 경우가 있어 안전하게 eager 권장
            attn_implementation="eager",
        )

        mode = "gpt-oss / MXFP4"
        if gpt_oss_dequantize_to_bf16:
            mode += " (dequantize->bf16)"

    else:
        bnb_cfg = _make_bnb_nf4_4bit_config()

        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map=device_map,
            trust_remote_code=trust_remote_code,
            quantization_config=bnb_cfg,
            torch_dtype=torch.bfloat16,
            attn_implementation="sdpa",
        )

        # QLoRA/4bit 학습 준비(가능할 때만)
        if prepare_model_for_kbit_training is not None:
            model = prepare_model_for_kbit_training(model)
        mode = "bnb / NF4 4bit (QLoRA-ready)"

    # 공통: 학습 준비 옵션(권장)
    if enable_gradient_checkpointing and hasattr(model, "gradient_checkpointing_enable"):
        model.gradient_checkpointing_enable()
    if hasattr(model.config, "use_cache"):
        model.config.use_cache = False  # 학습 시 권장

    # tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=trust_remote_code)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer, {"mode": mode}

In [4]:
# gpt-oss-20b 로드 시
model_id = "openai/gpt-oss-20b"
#model_id = "Qwen/Qwen3-14B"

# 1. gpt-oss-20b: MXFP4 네이티브 4bit
model, tokenizer, info = load_model_and_tokenizer(
    model_id, 
    gpt_oss_dequantize_to_bf16=False
)

# 2. gpt-oss-20b: BF16로 디양자화(학습 파이프라인 맞출 때)
# model, tokenizer, info = load_model_and_tokenizer(model_id, gpt_oss_dequantize_to_bf16=True)

print(info["mode"])
print("model dtype:", getattr(model, "dtype", None))
print(f"모델 로드 완료: {model.get_memory_footprint() / 1e9:.2f} GB")

`torch_dtype` is deprecated! Use `dtype` instead!
MXFP4 quantization requires Triton and kernels installed: CUDA requires Triton >= 3.4.0, XPU requires Triton >= 3.5.0, we will default to dequantizing the model to bf16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

gpt-oss / MXFP4
model dtype: torch.bfloat16
모델 로드 완료: 41.83 GB


## 3. LoRA 어댑터 설정

모델의 표현력을 최대한 유지하면서 학습하기 위해 모든 선형 레이어(`Linear`)에 LoRA를 적용한다.

In [5]:
# LoRA 설정
lora_config = LoraConfig(
    r=16,                       # Rank
    lora_alpha=32,              # Alpha
    target_modules=[            # 모든 Linear 레이어 타겟팅
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 모델에 LoRA 적용
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 7,962,624 || all params: 20,922,719,808 || trainable%: 0.0381


## 4. 도메인 데이터셋 준비 (금융)

이번 챕터에서는 일반적인 대화가 아닌, 금융 지식을 학습시키기 위해 `gbharti/finance-alpaca` 데이터셋을 사용한다.

In [6]:
# 금융 데이터셋 로드
dataset = load_dataset("gbharti/finance-alpaca", split="train[:1000]")

def format_finance_instruction(example):
    """
    금융 데이터셋을 프롬프트 형식으로 변환하는 함수다.
    """
    instruction = example['instruction']
    input_text = example.get('input', '')
    output = example['output']

    # 금융 전문가 페르소나 주입 (선택 사항)
    prefix = "당신은 금융 전문가 AI 어시스턴트입니다. 다음 질문에 전문적으로 답변하세요."

    if input_text:
        text = f"""### Instruction:
{prefix}
{instruction}

### Input:
{input_text}

### Response:
{output}"""
    else:
        text = f"""### Instruction:
{prefix}
{instruction}

### Response:
{output}"""

    return {"text": text}

# 데이터 포맷팅
formatted_dataset = dataset.map(format_finance_instruction)

print(f"금융 데이터셋 준비 완료: {len(formatted_dataset)}개")
print(formatted_dataset[0]['text'][:300] + "...")

금융 데이터셋 준비 완료: 1000개
### Instruction:
당신은 금융 전문가 AI 어시스턴트입니다. 다음 질문에 전문적으로 답변하세요.
For a car, what scams can be plotted with 0% financing vs rebate?

### Response:
The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a p...


## 5. 학습 설정 및 실행

메모리 효율을 위해 `paged_adamw_8bit` 옵티마이저를 사용한다. 이는 GPU 메모리가 부족할 때 CPU RAM을 활용하여 OOM(Out of Memory) 오류를 방지하는 QLoRA 학습의 필수 요소다.

In [8]:
# SFT 학습 설정
training_args = SFTConfig(
    output_dir="./GPT-OSS-20B-Finance-QLoRA",

    # 학습 파라미터
    max_steps=100,                      # 실습용 (실제 학습 시 num_train_epochs 사용)
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,

    # 메모리 최적화 옵션
    optim="paged_adamw_8bit",           # 페이징 옵티마이저 (중요)
    fp16=False,
    bf16=True,                          # BF16 사용 권장

    # 로깅 및 저장
    logging_steps=10,
    save_strategy="steps",
    save_steps=50,

    # 데이터 설정
    #max_seq_length=512,
    dataset_text_field="text",
    packing=False
)

# Trainer 생성
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    args=training_args,
    processing_class=tokenizer,
)

print("금융 모델 QLoRA 학습 시작...")
trainer.train()
print("학습 완료!")

# 모델 저장
trainer.save_model("./GPT-OSS-20B-Financ-Final")

금융 모델 QLoRA 학습 시작...


wandb: Currently logged in as: kubwai to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,3.599500
20,2.510100
30,2.433200
40,2.354200
50,2.326000
60,2.340800
70,2.323400
80,2.261000
90,2.266000
100,2.305800


학습 완료!


## 6. 요약

QLoRA는 4-bit 양자화를 통해 메모리 사용량을 획기적으로 줄이면서도 LoRA 학습을 가능하게 하는 기술이다.
이번 챕터에서는 금융 도메인 데이터를 활용하여 특수 목적의 모델을 효율적으로 학습하는 방법을 실습했다.

**다음 챕터**: Chapter 08 - 도메인 적응 SFT에서는 금융 외에도 의료, 법률 등 다양한 도메인 데이터를 처리하는 전략과 시스템 프롬프트 활용법을 다룬다.